# ResponseGetData

> preferred response class for all API requests


In [ ]:
# | default_exp client.ResponseGetData

In [ ]:
# | exporti
# pylint: disable=no-member

import re
from bs4 import BeautifulSoup

from dataclasses import dataclass, field
from typing import Optional

import orjson

import asyncio
import requests
import httpx
import aiohttp

from fastcore.utils import patch_to
import domolibrary.client.DomoError as de

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import test_eq

# Response Error Classes

In [ ]:
#| export
class BlockedByVPN(de.DomoError):
    def __init__(
        self,
        domo_instance: Optional[str] = None,
        ip_address : str = None,
        function_name: str = "get_data"
    ):
        ip_address_str = f"from {ip_address}" if ip_address else ""
        message = f"request blocked {ip_address_str} - check VPN settings"

        super().__init__(message=message, domo_instance=domo_instance, function_name=function_name)


In [ ]:
# | export
API_Response = any


@dataclass
class ResponseGetData:
    """preferred response class for all API Requests"""

    status: int
    response: API_Response
    is_success: bool
    auth: dict = field(repr = False, default=None)

    def set_response(self, response):
        self.response = response

In [ ]:
rgd = ResponseGetData(status=200, response="test", is_success=True)
rgd

ResponseGetData(status=200, response='test', is_success=True)

In [ ]:
# | hide
test_eq(rgd.is_success, True)

# Classmethods from Response objects by library

This code base supports two API request libraries, `requests.request` (synchronous) and `aiohttp.ClientRequest` (asynchronous) this can be extended as new libraries emerge with different performance characteristics.


## Requests Library

In [ ]:
# | export
@patch_to(ResponseGetData, cls_method=True)
def _from_requests_response(
    cls, res: requests.Response  # requests response object
) -> ResponseGetData:
    """returns ResponseGetData"""

    # JSON responses
    if res.ok and "application/json" in res.headers.get("Content-Type", {}):
        return cls(status=res.status_code, response=res.json(), is_success=True)

    # default text responses
    elif res.ok:
        return cls(status=res.status_code, response=res.text, is_success=True)

    # errors
    return cls(status=res.status_code, response=res.reason, is_success=False)

In [ ]:
show_doc(ResponseGetData._from_requests_response)


---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/client/ResponseGetData.py#L58){target="_blank" style="float:right; font-size:smaller"}

### ResponseGetData._from_requests_response

>      ResponseGetData._from_requests_response (res:requests.models.Response)

returns ResponseGetData

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| res | Response | requests response object |
| **Returns** | **ResponseGetData** |  |

In [ ]:
# test _from_requests_response returns ResponseGetData class
import os
import requests

url = f"https://domo-dojo.domo.com/api/content/v2/authentication"

tokenHeaders = {"Content-Type": "application/json"}

body = {
    "method": "password",
    "emailAddress": "jae@onyxreporting.com",
    "password": os.environ["DOJO_PASSWORD"],
}

res = requests.request(method="POST", url=url, json=body, headers=tokenHeaders)

print(res.status_code)

test_res = ResponseGetData._from_requests_response(res)
test_res.__dict__.keys()

200


dict_keys(['status', 'response', 'is_success', 'auth'])

In [ ]:
# | hide
test_eq(isinstance(test_res, ResponseGetData), True)

## HTTPX Library

In [ ]:
#| export
def find_ip(html,   html_tag: str = 'p'):
    ip_address_regex = r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
    soup = BeautifulSoup(html, 'html.parser')

    return re.findall(ip_address_regex, str(soup.find(html_tag)))[0]
                                 

In [ ]:
# | export
@patch_to(ResponseGetData, cls_method=True)
def _from_httpx_response(
    cls, res: requests.Response,  # requests response object
    auth : Optional[any] = None,
) -> ResponseGetData:
    """returns ResponseGetData"""


    # JSON responses
    ok = True if res.status_code <= 399 and res.status_code >= 200 else False

    if ok and '<title>Domo - Blocked</title>' in res.text:
        ip_address = find_ip(res.text)
        
        raise BlockedByVPN(auth.domo_instance, ip_address)
    
    if ok and "application/json" in res.headers.get("Content-Type", {}):
        try:
            return cls(status=res.status_code, response=res.json(), is_success=True, auth = auth)

        except Exception as e:
            return cls(status=res.status_code, response=res.text, is_success=True, auth=auth)

    # default text responses
    elif ok:
        return cls(status=res.status_code, response=res.text, is_success=True, auth = auth)

    # errors
    return cls(status=res.status_code, response=res.reason_phrase, is_success=False, auth=auth)


## Asyncio Response Handling

In [ ]:
# | export
STREAM_FILE_PATH = '__large-file.json'

async def _write_stream(res: httpx.Response,
                        file_name: str = STREAM_FILE_PATH,
                        stream_chunks=10):
    
    print(type(res), type(res.content), stream_chunks)

    index = 0
    with open(file_name, 'wb') as fd:
        async for chunk in res.content.iter_chunked(1024):
            index +=1
            print(f"writing chunk - {index}")
            fd.write(chunk)

            print(res.content.at_eof())
        
    print('done writing stream')
    
    return None


async def _read_stream(file_name : str):
    with open(file_name, "rb") as f:
        return f.read()


In [ ]:
# | export
@patch_to(ResponseGetData, cls_method=True)
async def _from_aiohttp_response(
    cls: ResponseGetData, 
    res: aiohttp.ClientResponse,  # requests response object
    auth : Optional[any] = None,
    process_stream: bool = False,
    stream_chunks : int = 10,
    debug_api : bool = False,
    response_file_name: str = None
) -> ResponseGetData:

    """async method returns ResponseGetData"""
    if debug_api:
        print( f"ResponseGetData: res.ok = {res.ok} , res.status = {res.status}" )
    

    try:
        data = None

        if process_stream:
            await _write_stream(res = res, stream_chunks = stream_chunks)
            data = await _read_stream(response_file_name)
        
        else:        
            data = await res.text()
    
        if debug_api:
            print('converting to text complete')
    
    except asyncio.TimeoutError as e:
        print(f"ResponseGetDataError: {str(e)} , trying content.read")

        data = await res.content.read()
    

    if res.ok and "application/json" in res.headers.get("Content-Type", {}):
        try:
            return cls(status=res.status, response= orjson.loads(data), is_success=True, auth = auth)
        except Exception as e:
            return cls(status=res.status, response=data, is_success=True, auth=auth)

    elif res.ok:
        return cls(status=res.status, response= data, is_success=True, auth = auth)

    # response is error
    else:
        return cls(status=res.status, response=res.reason, is_success=False, auth = auth)



In [ ]:
show_doc(ResponseGetData._from_aiohttp_response)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/client/ResponseGetData.py#L144){target="_blank" style="float:right; font-size:smaller"}

### ResponseGetData._from_aiohttp_response

>      ResponseGetData._from_aiohttp_response
>                                              (res:aiohttp.client_reqrep.Client
>                                              Response, auth:Optional[<built-
>                                              infunctionany>]=None,
>                                              process_stream:bool=False,
>                                              stream_chunks:int=10,
>                                              debug_api:bool=False,
>                                              response_file_name:str=None)

async method returns ResponseGetData

In [ ]:
# test _from_aiohttp_response returns ResponseGetData class

import os
import requests

url = f"https://domo-dojo.domo.com/api/content/v2/authentication"

body = {
    "method": "password",
    "emailAddress": "jae@onyxreporting.com",
    "password": os.environ["DOJO_PASSWORD"],
}

session = aiohttp.ClientSession()
res = await session.post(url=url, json=body)
await session.close()

print(res.status)

test_res = await ResponseGetData._from_aiohttp_response(res)
test_res.__dict__.keys()

200


dict_keys(['status', 'response', 'is_success', 'auth'])

In [ ]:
# | hide
test_eq(isinstance(test_res, ResponseGetData), True)

In [ ]:
# | export
@patch_to(ResponseGetData, cls_method=True)
async def _from_looper(cls: ResponseGetData,
                       res: ResponseGetData,  # requests response object
                       array: list
                       ) -> ResponseGetData:

    """async method returns ResponseGetData"""

    if res.is_success:
        res.response = array
        return res

    # response is error
    else:
        return res


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()